# Import des packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 84.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 21.0 MB/s eta 0:00:00


# Import des différentes bases de données
## 1ère base : base Ameli, contient les données des pathologies en France
Les données sont celles de l'Assurance Maladie du 31/12/2015 au 31/12/2021. On retrouve des variables classiques de structure de la population (à noter que cla_age_5 correspond à la classe d’âge par tranche de 5 ans) et les variables suivantes :
* patho_niv1: groupe de pathologies 
* patho_niv2: sous-groupe de pathologies
* patho_niv3: sous-groupe détaillé de pathologies
* top: libellé technique de la pathologie
* Ntop: effectif de patients pris en charge pour la pathologie dont il est question
* Npop: population de référence
* prev: prévalence de patients, ou proportion de patients, pris en charge pour la pathologie dont il est question.
* Niveau prioritaire : niveau de gravité de la maladie, noté de 1 (faible) à 3(fort).

<b> Première tentative d'import de la base. 
Il s'est avéré que c'était trop long quand bien même on augmente considérablement les ressources disponibles, c'est pourquoi nous sommes parvenus à cette solution qui n'est certes pas la meilleure mais qui permet au moins de faire tourner la base. Il faut la télécharger au préalable et la coller en local. </b>

test_1 = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/5f71ba43-afc8-43a0-b306-dafe29940f9c", sep = ";", encoding = 'utf-8', engine = 'python')
le lien du dossier data.gouv : 
https://www.data.gouv.fr/fr/datasets/effectif-de-patients-par-pathologie-sexe-classe-dage-et-territoire-departement-region/
methode "classique" pour obtenir un dataset a partir d un lien 
probleme d encodage d ou les parametres sep, encoding, engine


In [3]:
test_2 = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')

data_gen = test_2.dropna()

On choisit de 'simplement' supprimer les valeurs manquantes car, avec quatre millions d'observations il était parfois difficile de faire tourner certaines opérations. Par ailleurs, une fois qu'on supprime tous les NA, on garde tout de même deux millions d'observations ce qui reste très important. 

## 2ème base : la base détaillant les origines de la pollution
Les données de la table base_pollution.csv datent de 2016. Les variables de la  table sont celles utilisées pour calculer le potentiel ou pouvoir de réchauffement global (PRG). C'est un indicateur qui vise à regrouper sous une seule valeur l'effet additionné de toutes les substances contribuant à l'accroissement de l'effet de serre. Les variables recouvrent les principales activités humaines à l'origine de la pollution atmosphérique.

Nous présummons que l'unité des données de la base est la tonne de CO2.

Nous croisons cette base avec notre base principale car la qualité de l'air a des conséquences non négligeables sur la santé des agents. En effet, la pollution de l’air provoque à elle seule près de 6,5 millions de décès chaque année, soit un décès sur huit, ce qui la positionne parmi les principaux risques pour la santé dans le monde (source : https://www.atmo-france.org).

In [48]:
data_pollution = pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert", encoding = 'utf-8')

#Dictionnaire associant les numéros de département à ceux des régions

correspondance_reg = {
    84: ['01', '03', '07', '15', '26', '38', '42', '43', '63', '69', '73', '74'],  # Auvergne-Rhône-Alpes
    27: ['21', '25', '39', '58', '70', '71', '89', '90'],  # Bourgogne-Franche-Comté
    53: ['22', '29', '35', '56'],  # Bretagne
    28: ['18', '28', '36', '37', '41', '45'],  # Centre-Val de Loire
    94: ['2A', '2B'],  # Corse
    44: ['08', '10', '51', '52', '54', '55', '57', '67', '68', '88'],  # Grand Est
    32: ['02', '59', '60', '62', '80'],  # Hauts-de-France
    11: ['75', '77', '78', '91', '92', '93', '94', '95'],  # Île-de-France
    76: ['14', '27', '50', '61', '76'],  # Normandie
    75: ['16', '17', '19', '23', '24', '33', '40', '47', '64', '79', '86', '87'],  # Nouvelle-Aquitaine
    24: ['09', '11', '12', '30', '31', '32', '34', '46', '48', '65', '66', '81', '82'],  # Occitanie
    52: ['44', '49', '53', '72', '85'],  # Pays de la Loire
    93: ['04', '05', '06', '13', '83', '84']  # Provence-Alpes-Côte d'Azur
}

correspondance_reg = {dep: region for region, deps in correspondance_reg.items() for dep in deps}

#Création de la variable département à partir du code commune de l'INSEE
data_pollution['DEP'] = data_pollution['INSEE commune'].str[:2]

#DEP est de type serie car il ne contient pas que de variables numériques (2A et 2B) donc procédure un peu particulière
#Création de la variable région à partir de celle du département
size = len(data_pollution['DEP'])
data_pollution['REG'] = pd.Series([0]*size)

for i in range(size) :
    data_pollution.loc[i, 'REG'] = correspondance_reg.get(data_pollution['DEP'][i],"inconnu")

REG_DEP_na = data_pollution[(data_pollution['DEP'].isna()) | (data_pollution['REG'].isna())]

#On pensait d'abord récupérer les codes des départements et des régions grâce à une autre base mais on perdait des données
#Ce code a l'avantage de ne pas surcharger l'environnement de travail avec une base de donnnées suplémentaire

#Somme des niveaux de pollution par département
pollution_group_dep = data_pollution.drop(['Commune','INSEE commune', 'REG'], axis = 1).groupby('DEP', as_index=False).sum()

#Somme des niveaux de pollution par région
pollution_group_reg = data_pollution.drop(['Commune','INSEE commune', 'DEP'], axis = 1).groupby('REG', as_index = False).sum()


#Dictionnaire de correspondance des numéro de région avec leur nom
#Utile pour l'affichage des graphiques
nom_reg = {
    84: 'Auvergne-Rhône-Alpes',
    27: 'Bourgogne-Franche-Comté',
    53: 'Bretagne',
    28: 'Centre-Val de Loire',
    94: 'Corse',
    44: 'Grand Est',
    32: 'Hauts-de-France',
    11: 'Île-de-France',
    76: 'Normandie',
    75: 'Nouvelle-Aquitaine',
    24: 'Occitanie',
    52: 'Pays de la Loire',
    93: "Provence-Alpes-Côte d'Azur"
}

# Conversion de la colonne 'REG' en type int
pollution_group_reg['REG'] = pollution_group_reg['REG'].astype(int)

# Création de la variable 'Nom de la région' à partir de celle de la région
pollution_group_reg['Nom de la région'] = pollution_group_reg['REG'].map(nom_reg).fillna("inconnu")

# Réaranger l'ordre des colonnes
colonnes_ordre = ['REG', 'Nom de la région'] + [colonne for colonne in pollution_group_reg.columns if colonne not in ['REG', 'Nom de la région']]
pollution_group_reg = pollution_group_reg[colonnes_ordre]

# Importation des bases en local pour faire tourner le code main
data_pollution.to_csv('base_pollution.csv', sep=',', index=False, encoding='utf-8')
pollution_group_reg.to_csv('pollution_group_reg.csv', sep=',', index=False, encoding='utf-8')
pollution_group_dep.to_csv('pollution_group_dep.csv', sep=',', index=False, encoding='utf-8')

## 3ème base : base regroupant la population par département
Utilisée plus tard pour permettre de pondérer et de faire des groupes de départements les plus homogènes possibles.
Zozo je te laisse faire aussi pour celle-là.

In [5]:
url = 'https://www.insee.fr/fr/statistiques/fichier/1893198/estim-pop-dep-sexe-gca-1975-2023.xls'

noms_colonnes=['numero_dep','dep','total_habitants']
pop_test = pd.read_excel(url, sheet_name='2023',skiprows=3,header=1, usecols = [0,1,7], skipfooter=4, names=noms_colonnes)


#pop_test2 = pop_test.set_index(['Num_dep ', 'Nom_dep'])
#pop_test2

#pop_test3 = pop_test.drop(df.index[:3])
#print(pop_test3)

pop_test.to_csv('base_pop_dep.csv')

In [6]:
groupe_dep = pd.read_csv("pollution_group_dep.csv", encoding = 'utf-8')

In [7]:
print("nombre de categories differentes pour niv2 est :"+str(len(test_2['patho_niv2'].unique())))
print("nombre de categories differentes pour niv3 est :"+str(len(test_2['patho_niv3'].unique())))

nombre de categories differentes pour niv2 est :44
nombre de categories differentes pour niv3 est :57


In [8]:
liste_diff_patho1 = data_gen['patho_niv1'].unique()
liste_diff_patho2 = data_gen['patho_niv2'].unique()
liste_diff_patho3 = data_gen['patho_niv3'].unique()

print(liste_diff_patho1, "\n", liste_diff_patho2, "\n", liste_diff_patho3)

['Traitements du risque vasculaire (hors pathologies)'
 'Traitements psychotropes (hors pathologies)'
 'Séjours en hospitalisation complète pour prise en charge de la Covid-19'
 'Maladies respiratoires chroniques (hors mucoviscidose)'
 'Maternité (avec ou sans pathologies)' 'Maladies psychiatriques'
 'Autres affections de longue durée (dont 31 et 32)'
 'Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)'
 'Cancers' 'Diabète' 'Maladies cardio-neurovasculaires'
 'Maladies du foie ou du pancréas (hors mucoviscidose)'
 'Maladies inflammatoires ou rares ou VIH ou SIDA'
 'Insuffisance rénale chronique terminale'
 'Maladies neurologiques ou dégénératives'] 
 ['Traitements hypolipémiants (hors pathologies)'
 "Traitements antidépresseurs ou régulateurs de l'humeur (hors pathologies)"
 'Séjours en hospitalisation complète pour prise en charge de la Covid-19'
 'Maladies respiratoires chroniques (hors mucoviscidose)'
 'Maternité (avec ou sans pathologies)' 'Troubles 

# Preprocessing
## Création de variables et dataframes plus clairs pour manipulation et visualisation

In [9]:
#Creation d une colonne individus
data_gen.reset_index(inplace = True, names='ind') 

#creation d une colonne classe d age numerique pour que ce soit plus pratique

data_gen["classe_age"] = data_gen["cla_age_5"].apply(
    lambda x: np.mean(list(map(lambda y : int(y), x.split("-")))) if "-" in x else (np.nan if x == 'tsage' else 100)
)

/tmp/ipykernel_266/4051356911.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_gen["classe_age"] = data_gen["cla_age_5"].apply(


In [ ]:
#on cree un nouveau dataframe pour pouvoir faire des manipulations plus rapides sur le data_gen

In [10]:
df_model = data_gen[['ind','classe_age' , 'region', 'dept']]

In [ ]:
#l'autre base pour la pollution ne prend pas les départements d'outre-mer en compte
#il est donc probable que l'on soit contraints de les exclure de l'analyse 


In [11]:
df_model_test = pd.merge(df_model, groupe_dep, how = 'left', left_on='dept', right_on='DEP')

In [12]:
df_model_reduced = data_gen[['ind','classe_age' , 'region', 'dept']]

In [13]:
pd.get_dummies(data_gen, prefix = 'indicatrice_', columns=['patho_niv1'], dtype='int').dropna().to_csv('data_dummies.csv')

In [14]:
df_modelisation = pd.get_dummies(data_gen, prefix = 'indicatrice', columns=['patho_niv1'], dtype='int')
age_30_60 = [32, 37, 42, 47, 52, 57, 62] #utilise pour faire des etudes par classe d age plus ou moins precises
df_modelisation_30_60 = df_modelisation[df_modelisation['classe_age'].isin(age_30_60)]
df_modelisation_30_60 = df_modelisation_30_60[['classe_age'] + list(df_modelisation_30_60.columns.difference(['classe_age']))]

In [15]:
df_modelisation = df_modelisation.drop(['ind','patho_niv2', 'patho_niv3', 'top', 'cla_age_5', 'sexe', 'region', 'ntop', 'npop', 'prev', 'niveau_prioritaire', 'libelle_classe_age', 
                      'libelle_sexe', 'tri', 'annee'], axis = 1)

df_modelisation_30_60 = df_modelisation_30_60.drop(['ind','patho_niv2', 'patho_niv3', 'top', 'cla_age_5', 'sexe', 'region', 'ntop', 'npop', 'prev', 'niveau_prioritaire', 'libelle_classe_age', 
                      'libelle_sexe', 'tri', 'annee'], axis = 1)

In [16]:
#creation d un dictionnaire pour pouvoir avoir des classe d age qui sont des moyennes et pas la somme de
#tous les individus d'une même classe d age
#peut etre que ce n est pas la facon la plus synthetique de faire mais elle fonctionne
dico = {'classe_age' : 'mean',
       'indicatrice_Autres affections de longue durée (dont 31 et 32)' : 'sum',
       'indicatrice_Cancers' : 'sum', 'indicatrice_Diabète' : 'sum',
       'indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)' : 'sum',
       'indicatrice_Insuffisance rénale chronique terminale' : 'sum',
       'indicatrice_Maladies cardio-neurovasculaires' : 'sum',
       'indicatrice_Maladies du foie ou du pancréas (hors mucoviscidose)' : 'sum',
       'indicatrice_Maladies inflammatoires ou rares ou VIH ou SIDA' : 'sum',
       'indicatrice_Maladies neurologiques ou dégénératives' : 'sum',
       'indicatrice_Maladies psychiatriques' : 'sum',
       'indicatrice_Maladies respiratoires chroniques (hors mucoviscidose)' : 'sum',
       'indicatrice_Maternité (avec ou sans pathologies)' : 'sum',
       'indicatrice_Séjours en hospitalisation complète pour prise en charge de la Covid-19' : 'sum',
       'indicatrice_Traitements du risque vasculaire (hors pathologies)' : 'sum',
       'indicatrice_Traitements psychotropes (hors pathologies)' : 'sum'}


In [17]:
df_modelisation = df_modelisation.groupby('dept').agg(dico).reset_index()
df_modelisation.to_csv('base_groupe_dep_moy_age.csv')
df_modelisation_30_60 = df_modelisation_30_60.groupby('dept').agg(dico).reset_index()


## On a décidé de laisse de côté ces deux variables parce que ce sont les deux motifs d'hospitalisation les plus fréquents, et donc il n'y aurait eu que ces deux-là à analyser si on voulait voir les plus étudiés. 
## Le fait de les laisser permet de diversifier un peu plus l'analyse mais il convient de souligner qu'elles sont les motifs d'hospitalisation prépondérants. 

In [18]:
df_modelisation['plus_repandu'] = df_modelisation.drop(['dept', 'classe_age',
                                                        'indicatrice_Maladies cardio-neurovasculaires',
                                                        'indicatrice_Traitements psychotropes (hors pathologies)'], axis = 1).apply(lambda x: x.idxmax(), axis = 1)
df_modelisation_30_60['plus_repandu'] = df_modelisation_30_60.drop(['dept', 'classe_age',
                                                        'indicatrice_Maladies cardio-neurovasculaires',
                                                        'indicatrice_Traitements psychotropes (hors pathologies)'], axis = 1).apply(lambda x: x.idxmax(), axis = 1)


In [19]:
df_modelisation['plus_repandu'].unique()
#on aura donc peut-etre a considerer les variables avc et soins psychotropes
#differemment car elles risquent de peut-etre
#capter toute l analyse et ne pas laisser grand chose aux autres pathologies
#car dans le cas ou on les laisse dans le dataframe
#ce sont les deux seules maladies a apparaitre

array(['indicatrice_Maladies psychiatriques', 'indicatrice_Cancers',
       'indicatrice_Maladies neurologiques ou dégénératives',
       'indicatrice_Traitements du risque vasculaire (hors pathologies)'],
      dtype=object)

In [20]:
df_modelisation = df_modelisation.merge(pd.read_csv('base_pop_dep.csv'), how = 'left', left_on = 'dept', right_on = 'numero_dep')
df_modelisation_30_60 = df_modelisation_30_60.merge(pd.read_csv('base_pop_dep.csv'), how = 'left', left_on = 'dept', right_on = 'numero_dep')

In [21]:
df_modelisation.columns

Index(['dept', 'classe_age',
       'indicatrice_Autres affections de longue durée (dont 31 et 32)',
       'indicatrice_Cancers', 'indicatrice_Diabète',
       'indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)',
       'indicatrice_Insuffisance rénale chronique terminale',
       'indicatrice_Maladies cardio-neurovasculaires',
       'indicatrice_Maladies du foie ou du pancréas (hors mucoviscidose)',
       'indicatrice_Maladies inflammatoires ou rares ou VIH ou SIDA',
       'indicatrice_Maladies neurologiques ou dégénératives',
       'indicatrice_Maladies psychiatriques',
       'indicatrice_Maladies respiratoires chroniques (hors mucoviscidose)',
       'indicatrice_Maternité (avec ou sans pathologies)',
       'indicatrice_Séjours en hospitalisation complète pour prise en charge de la Covid-19',
       'indicatrice_Traitements du risque vasculaire (hors pathologies)',
       'indicatrice_Traitements psychotropes (hors pathologies)',
  

In [22]:
cols_pond = df_modelisation.columns.difference(['classe_age','dept','plus_repandu', 'Unnamed: 0',
                                                'numero_dep', 'dep', 'total_habitants'])
df_modelisation[cols_pond] = df_modelisation[cols_pond].div(df_modelisation['total_habitants'], axis=0)
cols_pond2 = df_modelisation_30_60.columns.difference(['classe_age','dept','plus_repandu', 'Unnamed: 0',
                                                'numero_dep', 'dep', 'total_habitants'])
df_modelisation_30_60[cols_pond2] = df_modelisation_30_60[cols_pond2].div(df_modelisation['total_habitants'], axis=0)

In [23]:
df_modelisation = df_modelisation.drop(['Unnamed: 0', 'plus_repandu', 'total_habitants'], axis = 1)
df_modelisation_30_60 = df_modelisation_30_60.drop(['Unnamed: 0', 'plus_repandu', 'total_habitants'], axis = 1)


In [24]:
base_pollution = pd.read_csv('base_pollution.csv')

In [25]:
base_pollution = base_pollution.groupby('DEP').sum()

In [26]:
base_pollution = base_pollution.drop(['INSEE commune', 'Commune', 'REG'], axis = 1)

In [27]:
base_pol_dep = df_modelisation.merge(base_pollution, how = 'right', left_on = 'dept', right_on = 'DEP',
                                     right_index=True)

base_pol_dep_30_60 = df_modelisation_30_60.merge(base_pollution, how = 'right', left_on = 'dept', right_on = 'DEP',
                                     right_index=True)

In [28]:
base_pol_dep_gen = base_pol_dep.drop(['numero_dep'], axis = 1)
base_pol_dep_gen_30_60 = base_pol_dep_30_60.drop(['numero_dep'], axis = 1)

In [29]:
num_cols = base_pol_dep_gen.select_dtypes(include=['number']).columns
num_data = base_pol_dep_gen[num_cols]

In [30]:
num_cols_30_60 = base_pol_dep_gen_30_60.select_dtypes(include=['number']).columns
num_data_30_60 = base_pol_dep_gen_30_60[num_cols_30_60]

In [31]:
scaler = StandardScaler()
data_standard = scaler.fit_transform(num_data)
data_standard_30_60 = scaler.fit_transform(num_data_30_60)

In [32]:
base_standard = pd.concat([base_pol_dep_gen[['dept', 'dep']], pd.DataFrame(data_standard, columns=num_cols)], axis=1)
base_standard_30_60 = pd.concat([base_pol_dep_gen_30_60[['dept', 'dep']], pd.DataFrame(data_standard_30_60, columns=num_cols_30_60)]
                                , axis=1)




In [33]:
#on perd indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité), 
#etrange car toutes les lignes valent 0 mais mis a part ca c est bien 

In [34]:
base_standard.to_csv("base_modelisation.csv")
base_standard_30_60.dropna().to_csv("base_modelisation_30_60.csv")

In [35]:
data_reg = data_gen.drop(['annee', 'patho_niv2', 'patho_niv3', 'top', 'cla_age_5', 'sexe', 'region',
                          'ntop', 'npop', 'prev', 'niveau_prioritaire','libelle_classe_age',
                          'libelle_sexe', 'tri'], axis = 1)

In [36]:
data_reg

,ind,patho_niv1,dept,classe_age
0,0,Traitements du risque vasculaire (hors patholo...,971,82.0
1,1,Traitements du risque vasculaire (hors patholo...,973,82.0
2,2,Traitements du risque vasculaire (hors patholo...,999,82.0
3,3,Traitements du risque vasculaire (hors patholo...,999,82.0
4,4,Traitements du risque vasculaire (hors patholo...,75,82.0
...,...,...,...,...
147764,276754,Maladies psychiatriques,999,12.0
147765,276755,Maladies psychiatriques,29,12.0
147766,276756,Maladies psychiatriques,16,12.0
147767,276757,Maladies psychiatriques,24,12.0


In [37]:
print(data_reg['classe_age'].isnull().sum())

9609


Étant donné que l'âge est une variable très importante, on ne peut pas garder des données dans lesquelles on trouve des NaN. 
On décide donc de supprimer toutes les lignes ne renseignant pas l'âge du patient. 

In [38]:
data_reg = data_reg.dropna()

In [39]:
data_reg = pd.get_dummies(data_reg, prefix = 'indicatrice', columns=['patho_niv1'], dtype='int')

In [40]:
data_reg.columns

Index(['ind', 'dept', 'classe_age',
       'indicatrice_Autres affections de longue durée (dont 31 et 32)',
       'indicatrice_Cancers', 'indicatrice_Diabète',
       'indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)',
       'indicatrice_Insuffisance rénale chronique terminale',
       'indicatrice_Maladies cardio-neurovasculaires',
       'indicatrice_Maladies du foie ou du pancréas (hors mucoviscidose)',
       'indicatrice_Maladies inflammatoires ou rares ou VIH ou SIDA',
       'indicatrice_Maladies neurologiques ou dégénératives',
       'indicatrice_Maladies psychiatriques',
       'indicatrice_Maladies respiratoires chroniques (hors mucoviscidose)',
       'indicatrice_Maternité (avec ou sans pathologies)',
       'indicatrice_Séjours en hospitalisation complète pour prise en charge de la Covid-19',
       'indicatrice_Traitements du risque vasculaire (hors pathologies)',
       'indicatrice_Traitements psychotropes (hors pathologie

In [41]:
data_reg = data_reg.drop(['indicatrice_Autres affections de longue durée (dont 31 et 32)',
                          'indicatrice_Diabète', 
                          'indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)',
                          'indicatrice_Insuffisance rénale chronique terminale',
                          'indicatrice_Maladies du foie ou du pancréas (hors mucoviscidose)',
                          'indicatrice_Maladies inflammatoires ou rares ou VIH ou SIDA',
                          'indicatrice_Maladies neurologiques ou dégénératives', 
                          'indicatrice_Maladies psychiatriques',
                          'indicatrice_Maternité (avec ou sans pathologies)',
                          'indicatrice_Séjours en hospitalisation complète pour prise en charge de la Covid-19',
                          'indicatrice_Traitements du risque vasculaire (hors pathologies)',
                          'indicatrice_Traitements psychotropes (hors pathologies)'], axis = 1)
#on ne garde que les variables qui peuvent nous interesser, 
#donc les maladies cardio-neurovasculaires, 
#les maladies respiratoires chroniques, 
#et les cancers

In [42]:
data_reg = data_reg.merge(base_pollution, how = 'left', left_on = 'dept', right_index = True)

In [43]:
data_reg = data_reg.dropna()

In [44]:
data_reg['transports/routier'] = data_reg['Autres transports'] + data_reg['Autres transports international']

In [45]:
data_reg = data_reg.drop(['Autres transports', 'Autres transports international'], axis = 1)

Sommer ces colonnes entre elles et les supprimer au niveau individuel ne semble pas être aberrant car les sources de pollution sont assez proches et exprimées dans la même unité. 

In [46]:
data_reg['cons_bat'] = data_reg['Résidentiel'] + data_reg['Tertiaire']

In [47]:
data_reg = data_reg.drop(['Résidentiel', 'Tertiaire'], axis = 1)

Même justification qu'avant et cela permettrait donc d'avoir moins de bruit pour faire des régressions.
D'autant que les corrélations sont vérifiées dans la partie cartographie/clustering.

In [ ]:
data_reg.to_csv("data_reg.csv")

In [ ]:
cancer = data_reg[data_reg['indicatrice_Cancers'] == 1]

In [ ]:
cancer['classe_age'].mean()

In [ ]:



classe_age_values = cancer['classe_age']

plt.figure(figsize=(8, 6))
sns.histplot(classe_age_values, bins=20, kde=True, color='skyblue')
plt.title('Distribution des âges auxquels un patient est hospitalisé pour cancer')
plt.xlabel("Classe d'âge")
plt.ylabel('Fréquence')
plt.grid(axis='y', alpha=0.75)
plt.show()